### 품사 태깅

In [103]:
# csv파일 불러옴
import csv
import re

raw = []
f = open(r'C:\Users\Yang\Desktop\project\ytn_article.csv', 'r', encoding='utf-8')
rdr = csv.reader(f)
for line in rdr:
    raw.append(line)
f.close()  

# 빈 text 제거 & 특수문자 제거
text = []
for i in range(len(raw)):
    if(raw[i][0] is not ' '):
        newtext = re.sub('[-=+,#/\?:^$@*\"~&%!<>()"]', '', raw[i][0])
        text.append(newtext)

In [2]:
# 품사 태깅(주어진 텍스트를 형태소 단위로 나누고 나눠진 형태소를 해당하는 품사와 함께 리스트화)
# stemming(단어들을 원형으로 포현), normalization(표현 방법이 다른 단어들을 통합시켜 같은 단어로 만듬)
import json
import os
from konlpy.tag import Okt

okt = Okt()

def tokenize(doc): # 형태소 분석기인 Okt를 이용해서 품사를 태깅시킴
    # norm은 정규화, stem은 근어로 표시하기를 나타냄
    return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]

os.chdir(r'C:\Users\Yang\Desktop\project') # 경로 설정

In [3]:
if os.path.isfile('text_docs.json'): # 경로에 text_docs.json 파일이 존재하면 json파일을 만들지 않고 load함
    with open('text_docs.json', encoding='utf-8') as f:
        text_docs = json.load(f)
else: # json파일이 없을 경우 text를 품사 태깅시킨 데이터로 변환시켜 json파일로 저장
    text_docs = [(tokenize(row)) for row in text]
    # JSON 파일로 저장
    with open('text_docs.json', 'w', encoding="utf-8") as make_file:
        json.dump(text_docs, make_file, ensure_ascii=False, indent="\t")

In [9]:
# 태깅된 각 형태소들을 하나의 list로 합침
tokens = [t for d in text_docs for t in d]
print(len(tokens))

1707035


In [14]:
import nltk
from pprint import pprint

nltk_text = nltk.Text(tokens, name='NMSC')

# 전체 토큰의 개수
total_tokens = nltk_text.tokens
print(len(total_tokens))

# 중복을 제외한 토큰의 개수
set_total_tokens = list(set(nltk_text.tokens))
print(len(set_total_tokens))            

# 출현 빈도가 높은 상위 토큰 10개
#pprint(nltk_text.vocab().most_common(50))

1707035
24065


### Word2vec

In [34]:
# 빈도수가 50개 이상인 token 확인.
most_common_tokens = nltk_text.vocab().most_common(len(set_total_tokens))
for i in range(len(set_total_tokens)):
    if(most_common_tokens[i][1]<50):
        index = i
        break
most_common_tokens_50 = most_common_tokens[:index]

In [4]:
# Word embedding(Word2vec 사용)
from gensim.models import Word2Vec
embedding_model = Word2Vec(text_docs, size=100, window = 2, min_count=50, workers=4, iter=100, sg=1)

In [21]:
for s in set_total_tokens:
    if("피해" in s):
        print(s)

피해망상/Noun
피해/Noun
피해자/Noun


In [34]:
# 두 벡터 사이의 코사인 유사도를 구함. 값이 클수록 비슷한 단어
word_similar = embedding_model.most_similar(positive=["피해/Noun"], topn=10)
#print(embedding_model.most_similar(positive=["피해/Noun"], topn=100))
print([i[0] for i in word_similar])

['인명/Noun', '침수/Noun', '재산/Noun', '사고/Noun', '화재/Noun', '규모/Noun', '농작물/Noun', '손해/Noun', '폐/Noun', '사망자/Noun']


C:\Users\Yang\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  
